In [1]:
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints


from neuralforecast import NeuralForecast
from neuralforecast.models import Informer
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, MSE, MAE

In [3]:
from numpy.random import seed
from random import randrange

In [4]:
df = pd.read_excel("SP100_vol (clean).xlsx", index_col="Date",parse_dates=True)
columns=df.columns
df_static=pd.read_excel("Static_df.xlsx")
n_series=len(df.columns)
train = df.iloc[:int(len(df['NKE'])*0.7)]
validation_length=int(len(df['NKE'])*0.7)-int(len(df['NKE'])*0.5)
train=train.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [5]:
n_inputs = [5,10,15,21,42,84]
hidden_size = [60,90,120,150,180]
epochs=[50,100,150,250,350,450,550,650,750]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']
losses=[MSE(),MAE(),MQLoss(level=[80, 90]),DistributionLoss(distribution='StudentT', level=[80, 90])]
n_head=[2,3,4,5,6]
decoder_input_size_multiplier=[0.3,0.4,0.5,0.6,0.7]
dropout=[0,0.2,0.3,0.4]
factor=[2,3,4]
conv_hidden_size=[12,22,32,42,52]
decoder_layers = [1,2,3]

# Hyperparameter Search

In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(5)
  m=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(3)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(3)
  model = Informer(h=1, input_size=n_inputs[i],
           decoder_input_size_multiplier=decoder_input_size_multiplier[a1], hidden_size=hidden_size[h],
           dropout=dropout[m], factor=factor[a2], n_head=n_head[a],
           conv_hidden_size=conv_hidden_size[a3],
           decoder_layers=decoder_layers[a5],
           loss=losses[m], max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
           val_check_steps=500,
           batch_size=32,
           scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Informer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Informer"].values
  else:
    Y_hat=forecasts["Informer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {hidden_size[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | factor: {factor[a2]} | decoder_layers: {decoder_layers[a5]}")
  print(f"Learning Rate: {n} | decoder_input_size_multiplier: {decoder_input_size_multiplier[a1]} | Dropout: {dropout[m]} | n_head: {n_head[a]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 58088380


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 350 | Inputs: 3 | hidden_size: 60 | Loss: MAE() | Scaler Type: robust | factor: 3 | decoder_layers: 2
Learning Rate: 2 | decoder_input_size_multiplier: 0.3 | Dropout: 0.2 | n_head: 4
RMSE: 0.0026923646923624396 | QLIKE: 0.02562070503979714 | MAE: 0.0015801182520588744 | MAPE: 0.1527446520741483


In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(5)
  m=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(3)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(3)
  model = Informer(h=1, input_size=n_inputs[i],
           decoder_input_size_multiplier=decoder_input_size_multiplier[a1], hidden_size=hidden_size[h],
           dropout=dropout[m], factor=factor[a2], n_head=n_head[a],
           conv_hidden_size=conv_hidden_size[a3],
           decoder_layers=decoder_layers[a5],
           loss=losses[m], max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
           val_check_steps=500,
           batch_size=32,
           scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Informer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Informer"].values
  else:
    Y_hat=forecasts["Informer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {hidden_size[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | factor: {factor[a2]} | decoder_layers: {decoder_layers[a5]}")
  print(f"Learning Rate: {n} | decoder_input_size_multiplier: {decoder_input_size_multiplier[a1]} | Dropout: {dropout[m]} | n_head: {n_head[a]} | encoder_layers: {encoder_layers[a4]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 99692143


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 550 | Inputs: 15 | hidden_size: 90 | Loss: DistributionLoss() | Scaler Type: standard | factor: 3 | decoder_layers: 1
Learning Rate: 0 | decoder_input_size_multiplier: 0.4 | Dropout: 0.4 | n_head: 6 | encoder_layers: 1
RMSE: 0.002555410868170968 | QLIKE: 0.02348263351971576 | MAE: 0.0014879118305409647 | MAPE: 0.14372182774158154


In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(5)
  m=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(3)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(3)
  model = Informer(h=1, input_size=n_inputs[i],
           decoder_input_size_multiplier=decoder_input_size_multiplier[a1], hidden_size=hidden_size[h],
           dropout=dropout[m], factor=factor[a2], n_head=n_head[a],
           conv_hidden_size=conv_hidden_size[a3],
           decoder_layers=decoder_layers[a5],
           loss=losses[m], max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
           val_check_steps=500,
           batch_size=32,
           scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Informer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Informer"].values
  else:
    Y_hat=forecasts["Informer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {hidden_size[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | factor: {factor[a2]} | decoder_layers: {decoder_layers[a5]}")
  print(f"Learning Rate: {n} | decoder_input_size_multiplier: {decoder_input_size_multiplier[a1]} | Dropout: {dropout[m]} | n_head: {n_head[a]} | encoder_layers: {encoder_layers[a4]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 37525783


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 750 | Inputs: 84 | hidden_size: 60 | Loss: MQLoss() | Scaler Type: minmax | factor: 2 | decoder_layers: 3
Learning Rate: 2 | decoder_input_size_multiplier: 0.4 | Dropout: 0.3 | n_head: 6 | encoder_layers: 3
RMSE: 0.003627676931846737 | QLIKE: 0.04991559403697488 | MAE: 0.002337886097000056 | MAPE: 0.22782533845173578


In [ ]:
for z in range (5):
  i=randrange(6)
  h=randrange(5)
  m=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(3)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(3)
  model = Informer(h=1, input_size=n_inputs[i],
           decoder_input_size_multiplier=decoder_input_size_multiplier[a1], hidden_size=hidden_size[h],
           dropout=dropout[m], factor=factor[a2], n_head=n_head[a],
           conv_hidden_size=conv_hidden_size[a3],
           decoder_layers=decoder_layers[a5],
           loss=losses[m], max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
           val_check_steps=500,
           batch_size=32,
           scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Informer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Informer"].values
  else:
    Y_hat=forecasts["Informer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {hidden_size[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | factor: {factor[a2]} | decoder_layers: {decoder_layers[a5]}")
  print(f"Learning Rate: {n} | decoder_input_size_multiplier: {decoder_input_size_multiplier[a1]} | Dropout: {dropout[m]} | n_head: {n_head[a]} | encoder_layers: {encoder_layers[a4]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 64785404


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 350 | Inputs: 15 | hidden_size: 180 | Loss: MQLoss() | Scaler Type: minmax | factor: 2 | decoder_layers: 2
Learning Rate: 3 | decoder_input_size_multiplier: 0.3 | Dropout: 0.3 | n_head: 2 | encoder_layers: 4
RMSE: 0.0032106023221640145 | QLIKE: 0.026319196347376797 | MAE: 0.002253339316871675 | MAPE: 0.24286359049663542


In [ ]:
for z in range (5):
  i=randrange(6)
  h=randrange(5)
  m=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(3)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(3)
  model = Informer(h=1, input_size=n_inputs[i],
           decoder_input_size_multiplier=decoder_input_size_multiplier[a1], hidden_size=hidden_size[h],
           dropout=dropout[m], factor=factor[a2], n_head=n_head[a],
           conv_hidden_size=conv_hidden_size[a3],
           decoder_layers=decoder_layers[a5],
           loss=losses[m], max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
           val_check_steps=500,
           batch_size=32,
           scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Informer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Informer"].values
  else:
    Y_hat=forecasts["Informer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {hidden_size[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | factor: {factor[a2]} | decoder_layers: {decoder_layers[a5]}")
  print(f"Learning Rate: {n} | decoder_input_size_multiplier: {decoder_input_size_multiplier[a1]} | Dropout: {dropout[m]} | n_head: {n_head[a]} | encoder_layers: {encoder_layers[a4]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 7901981


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 650 | Inputs: 21 | hidden_size: 180 | Loss: MQLoss() | Scaler Type: robust | factor: 3 | decoder_layers: 2
Learning Rate: 1 | decoder_input_size_multiplier: 0.6 | Dropout: 0.3 | n_head: 5 | encoder_layers: 3
RMSE: 0.0026917350837004465 | QLIKE: 0.025490133730841417 | MAE: 0.0015636787452678468 | MAPE: 0.14852668667703187


In [ ]:
for z in range (5):
  i=randrange(6)
  h=randrange(5)
  m=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(3)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(3)
  model = Informer(h=1, input_size=n_inputs[i],
           decoder_input_size_multiplier=decoder_input_size_multiplier[a1], hidden_size=hidden_size[h],
           dropout=dropout[m], factor=factor[a2], n_head=n_head[a],
           conv_hidden_size=conv_hidden_size[a3],
           decoder_layers=decoder_layers[a5],
           loss=losses[m], max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
           val_check_steps=500,
           batch_size=32,
           scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Informer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Informer"].values
  else:
    Y_hat=forecasts["Informer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {hidden_size[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | factor: {factor[a2]} | decoder_layers: {decoder_layers[a5]}")
  print(f"Learning Rate: {n} | decoder_input_size_multiplier: {decoder_input_size_multiplier[a1]} | Dropout: {dropout[m]} | n_head: {n_head[a]} | encoder_layers: {encoder_layers[a4]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 99911194


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 50 | Inputs: 3 | hidden_size: 120 | Loss: MQLoss() | Scaler Type: robust | factor: 2 | decoder_layers: 2
Learning Rate: 1 | decoder_input_size_multiplier: 0.4 | Dropout: 0.3 | n_head: 6 | encoder_layers: 3
RMSE: 0.0027231367573523624 | QLIKE: 0.026166193285566935 | MAE: 0.0015726901108348863 | MAPE: 0.15295641364955564


In [ ]:
for z in range (5):
  i=randrange(6)
  h=randrange(5)
  m=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(3)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(3)
  model = Informer(h=1, input_size=n_inputs[i],
           decoder_input_size_multiplier=decoder_input_size_multiplier[a1], hidden_size=hidden_size[h],
           dropout=dropout[m], factor=factor[a2], n_head=n_head[a],
           conv_hidden_size=conv_hidden_size[a3],
           decoder_layers=decoder_layers[a5],
           loss=losses[m], max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
           val_check_steps=500,
           batch_size=32,
           scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Informer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Informer"].values
  else:
    Y_hat=forecasts["Informer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {hidden_size[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | factor: {factor[a2]} | decoder_layers: {decoder_layers[a5]}")
  print(f"Learning Rate: {n} | decoder_input_size_multiplier: {decoder_input_size_multiplier[a1]} | Dropout: {dropout[m]} | n_head: {n_head[a]} | encoder_layers: {encoder_layers[a4]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 84661054


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 650 | Inputs: 21 | hidden_size: 180 | Loss: MQLoss() | Scaler Type: robust | factor: 4 | decoder_layers: 3
Learning Rate: 3 | decoder_input_size_multiplier: 0.4 | Dropout: 0.3 | n_head: 4 | encoder_layers: 3
RMSE: 0.0028556931945265484 | QLIKE: 0.02980722723669842 | MAE: 0.001670739529996997 | MAPE: 0.15718944124070136


In [ ]:
for z in range (5):
  i=randrange(6)
  h=randrange(5)
  m=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(3)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(3)
  model = Informer(h=1, input_size=n_inputs[i],
           decoder_input_size_multiplier=decoder_input_size_multiplier[a1], hidden_size=hidden_size[h],
           dropout=dropout[m], factor=factor[a2], n_head=n_head[a],
           conv_hidden_size=conv_hidden_size[a3],
           decoder_layers=decoder_layers[a5],
           loss=losses[m], max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
           val_check_steps=500,
           batch_size=32,
           scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Informer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Informer"].values
  else:
    Y_hat=forecasts["Informer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {hidden_size[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | factor: {factor[a2]} | decoder_layers: {decoder_layers[a5]}")
  print(f"Learning Rate: {n} | decoder_input_size_multiplier: {decoder_input_size_multiplier[a1]} | Dropout: {dropout[m]} | n_head: {n_head[a]} | encoder_layers: {encoder_layers[a4]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 53508068


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 150 | Inputs: 3 | hidden_size: 180 | Loss: MAE() | Scaler Type: standard | factor: 2 | decoder_layers: 3
Learning Rate: 2 | decoder_input_size_multiplier: 0.6 | Dropout: 0.2 | n_head: 6 | encoder_layers: 3
RMSE: 0.0025671840600963274 | QLIKE: 0.023870203355431007 | MAE: 0.0014796783554983245 | MAPE: 0.14253602004293936


INFO:lightning_fabric.utilities.seed:Seed set to 52829811

Epoch: 550 | Inputs: 15 | hidden_size: 90 | Loss: DistributionLoss() | Scaler Type: standard | factor: 3 | decoder_layers: 1
Learning Rate: 0 | decoder_input_size_multiplier: 0.4 | Dropout: 0.4 | n_head: 6 | encoder_layers: 1
RMSE: 0.002555410868170968 | QLIKE: 0.02348263351971576 | MAE: 0.0014879118305409647 | MAPE: 0.14372182774158154

In [ ]:
for z in range (5):
  randomseed=randrange(129228148)
  model = Informer(h=1, input_size=15,
           decoder_input_size_multiplier=0.4, hidden_size=90,
           dropout=0.4, factor=3, n_head=6,
           conv_hidden_size=32,
           decoder_layers=3,
           loss=losses[3], max_steps=550,
           learning_rate=learning_rate[0], num_lr_decays=num_lr_decays[0],
           val_check_steps=500,
           batch_size=32,
           scaler_type='standard',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Informer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Informer"].values
  else:
    Y_hat=forecasts["Informer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 103984860


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Random Seed: 53629349
RMSE: 0.0025598325178645322 | QLIKE: 0.02349754649858009 | MAE: 0.0014888334688729825 | MAPE: 0.14414679467578126


In [ ]:
for z in range (5):
  randomseed=randrange(129228148)
  model = Informer(h=1, input_size=15,
           decoder_input_size_multiplier=0.4, hidden_size=90,
           dropout=0.4, factor=3, n_head=6,
           conv_hidden_size=32,
           decoder_layers=3,
           loss=losses[3], max_steps=550,
           learning_rate=learning_rate[0], num_lr_decays=num_lr_decays[0],
           val_check_steps=500,
           batch_size=32,
           scaler_type='standard',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Informer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Informer"].values
  else:
    Y_hat=forecasts["Informer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 44695605


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Random Seed: 59706950
RMSE: 0.002560726021228924 | QLIKE: 0.02373156801313679 | MAE: 0.0014946974172928345 | MAPE: 0.1439976390391517


In [ ]:
for z in range (5):
  randomseed=randrange(129228148)
  model = Informer(h=1, input_size=15,
           decoder_input_size_multiplier=0.4, hidden_size=90,
           dropout=0.4, factor=3, n_head=6,
           conv_hidden_size=32,
           decoder_layers=3,
           loss=losses[3], max_steps=550,
           learning_rate=learning_rate[0], num_lr_decays=num_lr_decays[0],
           val_check_steps=500,
           batch_size=32,
           scaler_type='standard',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Informer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Informer"].values
  else:
    Y_hat=forecasts["Informer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 66767543


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Random Seed: 62771770
RMSE: 0.00255124626599572 | QLIKE: 0.02316486949913828 | MAE: 0.0014987311861506406 | MAPE: 0.14626835054351725


In [ ]:
for z in range (5):
  randomseed=randrange(129228148)
  model = Informer(h=1, input_size=15,
           decoder_input_size_multiplier=0.4, hidden_size=90,
           dropout=0.4, factor=3, n_head=6,
           conv_hidden_size=32,
           decoder_layers=3,
           loss=losses[3], max_steps=550,
           learning_rate=learning_rate[0], num_lr_decays=num_lr_decays[0],
           val_check_steps=500,
           batch_size=32,
           scaler_type='standard',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Informer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Informer"].values
  else:
    Y_hat=forecasts["Informer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 64877732


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Random Seed: 50240754
RMSE: 0.0025647927281366102 | QLIKE: 0.023961074130152558 | MAE: 0.0014923820118654195 | MAPE: 0.14372477364011754


# Test

In [6]:
test_length=int(len(df['NKE'])*0.3)
df1=df.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

# H=1

In [ ]:
model = Informer(h=1, input_size=15,
           decoder_input_size_multiplier=0.4, hidden_size=90,
           dropout=0.4, factor=3, n_head=6,
           conv_hidden_size=32,
           decoder_layers=3,
           loss=losses[3], max_steps=550,
           learning_rate=learning_rate[0], num_lr_decays=num_lr_decays[0],
           val_check_steps=500,
           batch_size=32,
           scaler_type='standard',
              random_seed=91659253)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=1,static_df=df_static,n_windows=None, test_size=test_length-test_length%1,step_size=1)
forecasts = forecasts.dropna()
if "Informer-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["Informer"].values
else:
  Y_hat=forecasts["Informer-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 91659253


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.0038727352169593337 | QLIKE: 0.03171137741753872 | MAE: 0.0024056815164261052 | MAPE: 0.17953156767113432


In [ ]:
if "Informer-median" not in list(forecasts.columns.values):
  df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
  df2=df2.pivot(columns="unique_id")
else:
  df2=forecasts[["ds","unique_id","Informer-median"]].set_index("ds")
  df2=df2.pivot(columns="unique_id")

In [ ]:
df2.to_excel("Informer forecasts.xlsx")

# H=5

In [7]:
H=5

model = Informer(h=H, input_size=15,
           decoder_input_size_multiplier=0.4, hidden_size=90,
           dropout=0.4, factor=3, n_head=6,
           conv_hidden_size=32,
           decoder_layers=3,
           loss=losses[3], max_steps=550,
           learning_rate=learning_rate[0], num_lr_decays=num_lr_decays[0],
           val_check_steps=500,
           batch_size=32,
           scaler_type='standard',
              random_seed=91659253)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=H,static_df=df_static,n_windows=None, test_size=test_length-test_length%H,step_size=H)
forecasts = forecasts.dropna()
if "Informer-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["Informer"].values
else:
  Y_hat=forecasts["Informer-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

if "Informer-median" not in list(forecasts.columns.values):
  df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
  df2=df2.pivot(columns="unique_id")
else:
  df2=forecasts[["ds","unique_id","Informer-median"]].set_index("ds")
  df2=df2.pivot(columns="unique_id")

df2.to_excel("Informer forecasts.xlsx")

INFO:lightning_fabric.utilities.seed:Seed set to 91659253


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.004810647503862946 | QLIKE: 0.04621980546512753 | MAE: 0.0028530351156349065 | MAPE: 0.2075608985109403


# H=10

In [8]:
H=10

model = Informer(h=H, input_size=15,
           decoder_input_size_multiplier=0.4, hidden_size=90,
           dropout=0.4, factor=3, n_head=6,
           conv_hidden_size=32,
           decoder_layers=3,
           loss=losses[3], max_steps=550,
           learning_rate=learning_rate[0], num_lr_decays=num_lr_decays[0],
           val_check_steps=500,
           batch_size=32,
           scaler_type='standard',
              random_seed=91659253)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=H,static_df=df_static,n_windows=None, test_size=test_length-test_length%H,step_size=H)
forecasts = forecasts.dropna()
if "Informer-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["Informer"].values
else:
  Y_hat=forecasts["Informer-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

if "Informer-median" not in list(forecasts.columns.values):
  df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
  df2=df2.pivot(columns="unique_id")
else:
  df2=forecasts[["ds","unique_id","Informer-median"]].set_index("ds")
  df2=df2.pivot(columns="unique_id")

df2.to_excel("Informer forecasts.xlsx")

INFO:lightning_fabric.utilities.seed:Seed set to 91659253


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.005677534183098615 | QLIKE: 0.059876085667915614 | MAE: 0.0032839211901392867 | MAPE: 0.23310020083417615
